# Import

In [29]:
import io
import os
import missingno
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Load and explore the dataset, visualizing the results of the exploration.

In [30]:
# 1. Load the dataset
file_path = 'Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv'

try:
    # Load the data
    df = pd.read_csv(file_path)
    
    # Display the first 5 rows
    print("File loaded successfully!")
    #print(df.head())

except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")

File loaded successfully!


In [31]:
# 2. Filter for 'Production' (Element Code 5510) and unit 't' (tonnes)
# This isolates the data representing the physical quantity produced.
prod_df = df[(df['Element Code'] == 5510) & (df['Unit'] == 't')].copy()

In [32]:
# 3. FILTER: Exclusion of Animal-derived Items (Proxy for 'plant production')
# Define keywords to exclude (case-insensitive)
animal_keywords = [
    'Milk', 'Meat', 'Egg', 'Wool', 'Honey', 'Hides', 
    'Offal', 'Animals', 'Fats', 'Guts', 'Animal', 'Silkworm', 
    'Poultry', 'Carcass', 'Goat', 'Cattle', 'Sheep', 'Pig'
]

# Create an exclusion mask (True if it is an animal product)
# The '|' joins the keywords with an OR logic.
exclusion_mask = prod_df['Item'].str.contains('|'.join(animal_keywords), case=False, na=False)

# Apply the filter: keep only items NOT in the exclusion mask (i.e., plant production)
plant_prod_df = prod_df[~exclusion_mask].copy()

In [33]:
# 4. Identify the year columns
# Finds all columns starting with 'Y' and representing an integer year between 1961 and 2023
year_cols = [col for col in plant_prod_df.columns if col.startswith('Y') and col[1:].isdigit() and 1961 <= int(col[1:]) <= 2023]

# Convert the year values to numeric (coercing errors to NaN)
for col in year_cols:
    plant_prod_df[col] = pd.to_numeric(plant_prod_df[col], errors='coerce')

In [34]:
# 5. Calculate the total production (sum across all years) and group by Item
# Calculate the total production for each record (Area/Item/Element combination)
plant_prod_df['Total_Production'] = plant_prod_df[year_cols].sum(axis=1)

# Group by Item and sum the grand total production
total_production_by_plant_item = plant_prod_df.groupby('Item')['Total_Production'].sum().reset_index()

Select Top n plant-production

In [35]:
n = 15 # <--- Top n

In [36]:
# 6. Sort and select the Top n
top_n_plant_items = total_production_by_plant_item.sort_values(by='Total_Production', ascending=False).head(n)

In [37]:
# 7. Format the result for printing
top_n_items_formatted = top_n_plant_items.copy()
# Convert total tonnes to Billions of Tonnes for readability
top_n_items_formatted['Total_Production_Billion_Tonnes'] = top_n_items_formatted['Total_Production'] / 1e9
top_n_items_formatted = top_n_items_formatted[['Item', 'Total_Production_Billion_Tonnes']].round(2)
top_n_items_formatted.rename(columns={'Item': 'Plant Product', 'Total_Production_Billion_Tonnes': 'Total Production (Billion Tonnes)'}, inplace=True)

In [38]:
# Print the result
print("\n--- Top n Plant Products by Total Production (1961-2023) ---\n")
print(top_n_items_formatted.to_markdown(index=False))


--- Top n Plant Products by Total Production (1961-2023) ---

| Plant Product                  |   Total Production (Billion Tonnes) |
|:-------------------------------|------------------------------------:|
| Cereals, primary               |                              557.57 |
| Sugar Crops Primary            |                              387.89 |
| Sugar cane                     |                              314.85 |
| Roots and Tubers, Total        |                              195.11 |
| Vegetables Primary             |                              169.45 |
| Maize (corn)                   |                              166.25 |
| Rice                           |                              152.29 |
| Wheat                          |                              149.45 |
| Fruit Primary                  |                              148.27 |
| Potatoes                       |                               88.12 |
| Sugar beet                     |                           

In [39]:
top_n_Plant_Product = top_n_items_formatted['Plant Product'].to_list()

# Filter the DataFrame, keeping only the rows where the 'Item' column
# contains one of the values present in the 'valori_agricoli' list
df_filtered = df[df['Item'].isin(top_n_Plant_Product)]

# Check the result
print(f"Original rows: {len(df)}")
print(f"Rows after filtering: {len(df_filtered)}")
#print(df_filtered.head())
df = df_filtered

Original rows: 78164
Rows after filtering: 8698


In [40]:
# 1. Define the list of columns to drop
# We focus on removing redundant metadata columns (Codes, Flags, Domain info)
COLUMNS_TO_DROP = [
    'Area Code',
    'Item Code',
    'Element Code',
    'Year Code',
    'Flag',
    'Flag Description',
    'Symbol',
    'Domain Code',
    'Domain',
    
    #redundant
    'Area Code', 
    'Area Code (M49)',
    'Item Code', 
    'Item Code (CPC)',
    'Element Code'
    
]

# 2. Identify which columns from the list actually exist in the DataFrame (df_filtered)
# This prevents the code from throwing an error if some columns are missing.
existing_columns_to_drop = [col for col in COLUMNS_TO_DROP if col in df_filtered.columns]

# 3. Drop the columns and create the clean DataFrame
# axis=1 specifies that we are operating on columns
df_clean = df_filtered.drop(columns=existing_columns_to_drop, axis=1)

# 4. Check the result
print("Columns remaining after cleanup:")
print(df_clean.columns)
print("\nFirst 5 rows of the cleaned DataFrame:")
print(df_clean.head())
df = df_clean

Columns remaining after cleanup:
Index(['Area', 'Item', 'Element', 'Unit', 'Y1961', 'Y1962', 'Y1963', 'Y1964',
       'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972',
       'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980',
       'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987', 'Y1988',
       'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996',
       'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004',
       'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012',
       'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020',
       'Y2021', 'Y2022', 'Y2023'],
      dtype='object')

First 5 rows of the cleaned DataFrame:
           Area          Item         Element   Unit     Y1961     Y1962  \
13  Afghanistan        Barley  Area harvested     ha  350000.0  350000.0   
14  Afghanistan        Barley           Yield  kg/ha    1080.0    1080.0   
15  Afghani

In [41]:
df.columns

Index(['Area', 'Item', 'Element', 'Unit', 'Y1961', 'Y1962', 'Y1963', 'Y1964',
       'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971', 'Y1972',
       'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979', 'Y1980',
       'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987', 'Y1988',
       'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995', 'Y1996',
       'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003', 'Y2004',
       'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012',
       'Y2013', 'Y2014', 'Y2015', 'Y2016', 'Y2017', 'Y2018', 'Y2019', 'Y2020',
       'Y2021', 'Y2022', 'Y2023'],
      dtype='object')

In [42]:
# Assuming 'df' is your current wide DataFrame
# If you have any other metadata columns (e.g., 'Flag', 'Symbol') in 'df', 
# add them to the 'fixed_id_vars' list below to keep them.

fixed_id_vars = ['Area', 'Item', 'Element'
                 #, 'Unit' remove Unit
                 ] 
year_columns = [col for col in df.columns if col.startswith('Y')]

df_with_metadata = (
    df.melt(
        id_vars=fixed_id_vars,
        value_vars=year_columns,
        var_name='Year',
        value_name='Value'
    )
    .assign(Year=lambda x: x['Year'].str.replace('Y', '').astype(int))
    #.query("Element == 'Production'") # Still filtering rows to one variable
    # NOTE: The .drop(columns=['Element', 'Unit']) step has been REMOVED.
    .sort_values(by=['Area', 'Item', 'Year'])
    .reset_index(drop=True)
)

print("DataFrame including Element and Unit:")
print(df_with_metadata.head())

df = df_with_metadata

DataFrame including Element and Unit:
          Area    Item         Element  Year     Value
0  Afghanistan  Barley  Area harvested  1961  350000.0
1  Afghanistan  Barley           Yield  1961    1080.0
2  Afghanistan  Barley      Production  1961  378000.0
3  Afghanistan  Barley  Area harvested  1962  350000.0
4  Afghanistan  Barley           Yield  1962    1080.0


In [43]:
# --- 1. Define Paths ---
FOLDER_NAME = 'processed data'
FILE_NAME = 'analyzed_agricultural_data.csv' # CSV Extension
FULL_PATH = os.path.join(FOLDER_NAME, FILE_NAME)

# --- 2. Create the Folder ---
# Checks if the folder exists and creates it if it doesn't
if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)
    print(f"The folder '{FOLDER_NAME}' has been created.")

# --- 3. Save the DataFrame as CSV ---
# The to_csv() function OVERWRITES the existing file (the update mechanism).
df.to_csv(
    FULL_PATH, 
    index=False, # Do not save the Pandas row index
    encoding='utf-8' # Standard encoding to support all characters
)

print(f"\nFile '{FILE_NAME}' updated and saved to: {FULL_PATH}")


File 'analyzed_agricultural_data.csv' updated and saved to: processed data\analyzed_agricultural_data.csv


In [ ]:
file_name = "Production_Crops_Livestock_E_All_Data_NOFLAG.csv"

# DEFINITION OF FOLDER STRUCTURE
# FOLDER_NAME = 'processed data'
parent_folder =  FOLDER_NAME # Parent folder (Outer folder)
# Sub-folder (Inner folder where files will actually go)
sub_folder = "processed_fao_data"

# Combine them to get the full path: "FAO_Project_Output/processed_fao_data"
output_dir = os.path.join(parent_folder, sub_folder)

# ==========================================
# CREATE NESTED DIRECTORY STRUCTURE
# ==========================================
try:
    # os.makedirs creates all intermediate directories needed
    os.makedirs(output_dir, exist_ok=True)
    print(f"Directory structure created: {output_dir}")
except OSError as e:
    print(f"Error creating directory {output_dir}: {e}")
    exit()

# ==========================================
# FILTER AND SAVE SEPARATE DATASETS
# ==========================================
filters = [
    ('Production', 'production_tonnes.csv'),
    ('Area harvested', 'area_harvested_ha.csv'),
    ('Yield', 'yield_kgha.csv')
]

print("Filtering and saving files into the sub-folder...")

for element, filename in filters:
    # Apply filter
    df_filtered = df[df['Element'] == element].copy()
    
    # Create the full file path inside the nested folder
    filepath = os.path.join(output_dir, filename)
    
    if not df_filtered.empty:
        df_filtered.drop(columns=['Element']).to_csv(filepath, index=False)
        print(f" -> Saved: {filepath}")
    else:
        print(f" -> Warning: No data found for {element}).")

print("\nProcessing complete.")

Directory structure created: processed data\processed_fao_data
Filtering and saving files into the sub-folder...
 -> Saved: processed data\processed_fao_data\production_tonnes.csv
 -> Saved: processed data\processed_fao_data\area_harvested_ha.csv
 -> Saved: processed data\processed_fao_data\yield_kgha.csv

Processing complete.
